In [ ]:
Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

In [ ]:
#Object Detection transfer learning training using tensorflow mobilenetssdlitev2 model on Azure ML

In [ ]:
#Build tensorflow object detection docker image for AzureML
#Specify your docker container registry (<ACR_NAME>.azurecr.io) and the repo name (dw-tf-od2:v1). You can go to https://portal.azure.com and go to container registry to find your container details or create new one. 
import docker
!docker build --tag <ACR_NAME>.azurecr.io/dw-tf-od2:v1 './docker'

In [ ]:
#!az login

In [ ]:
#Authenticate to container
!az acr login -n <ACR_NAME>

In [ ]:
#Push docker image to Azure container registry which will then be used for transfer learning training
!docker push <ACR_NAME>.azurecr.io/dw-tf-od2:v1

In [ ]:
#Setup workspace
import azureml.core
from azureml.core import Workspace
print(azureml.core.VERSION)

In [ ]:
import azureml.core
print("SDK version:", azureml.core.VERSION)

In [ ]:
'''from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics=True)
'''

In [ ]:
# Include your subscription, resource_group and the workspace details which will be used for experiment runs. This will create the workspace, if it doesn't exist
subscription_id = <subscriptionid>
resource_group = <resourcegroup>
workspace_name = <workspacename>
location=<loc>

# create aml workspace or create it azure portal
#https://docs.microsoft.com/en-us/python/api/overview/azure/ml/?view=azure-ml-py#workspace

from azureml.core import Workspace
ws = Workspace.create(name=workspace_name,
                      subscription_id=subscription_id,
                      resource_group=resource_group,
                      create_resource_group=True,
                      location=loc
                     )

In [ ]:
#Capture your Azure container registry details

acr_server = "<ACR_NAME>.azurecr.io"
acr_login = "<ACR_NAME>"
acr_pwd = "<PWD>"
#acr_repo_name = "dw-tf-od2:v1-gpu"
acr_repo_name = "dw-tf-od2:v1"

In [ ]:
#Reload workspace details for training

from azureml.core import Workspace

ws = Workspace.from_config()
ws.get_details()

In [ ]:
# setup datastore for loading custom labelled datasets
# For this training the data set was labelled using https://github.com/Microsoft/VoTT tool
ds = ws.get_default_datastore()
ds.name

In [ ]:
# Load data folder to default storage datastore
ds.upload(
    src_dir='./upload_data',
    target_path='tfdata',
    overwrite=True)

In [ ]:
"""
# train remote VM - gpu
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

try:
    compute_target = ComputeTarget(workspace=ws, name='dw-gpu')
    print('found existing:', compute_target.name)
except ComputeTargetException:
    print('creating new.')
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='STANDARD_NC6',
        min_nodes=0,
        max_nodes=1)
    compute_target = ComputeTarget.create(ws, 'dw-gpu', compute_config)
    compute_target.wait_for_completion(show_output=True)
"""

In [ ]:
# train remote VM - cpu
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

try:
    compute_target = ComputeTarget(workspace=ws, name='dw-cpu1')
    print('found existing:', compute_target.name)
except ComputeTargetException:
    print('creating new.')
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='STANDARD_D3_V2',
        min_nodes=0,
        max_nodes=1)
    compute_target = ComputeTarget.create(ws, 'dw-cpu1', compute_config)
    compute_target.wait_for_completion(show_output=True)


In [ ]:
# Mounting the uploaded data for training
from azureml.core import Datastore
from azureml.core.runconfig import DataReferenceConfiguration

ds = Datastore.get(ws, datastore_name="<DEFAULT WORKSPACEBLOBSTORE>")

dr_conf = DataReferenceConfiguration(
    datastore_name=ds.name,
    path_on_datastore='tfdata',
    #path_on_compute = '/tfdata'
    mode='mount') # or 'download'

In [ ]:
"""
# data reference using storage download (optional)
from azureml.core import Datastore
from azureml.core.runconfig import DataReferenceConfiguration

ds = Datastore.get(ws, datastore_name="<DEFAULT WORKSPACEBLOBSTORE>")

dr_conf = DataReferenceConfiguration(
    datastore_name=ds.name,
    path_on_datastore='tfdata',
    #path_on_compute = '/tfdata'
    mode='download') # or 'mount'
"""

In [ ]:
'''
# use estimator with download paramer (optional)
# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/training-with-deep-learning/distributed-cntk-with-custom-docker/distributed-cntk-with-custom-docker.ipynb
from azureml.train.estimator import Estimator

#params= {'--data_folder' : ds.as_download() }

params= {'--data_folder' : ds.as_download(),'--output_dir': './outputs'}

estimator = Estimator(source_directory='script',
                      compute_target=compute_target,
                      entry_script='train.py',
                      script_params=params,
                      node_count=1,
                      process_count_per_node=1,
                      pip_requirements_file = '../docker/requirements.txt', # pip packages
                      custom_docker_image='<ACR_NAME>.azurecr.io/dw-tf-od2:v1', # using public docker hub
                      use_gpu=False)
                      
'''

In [ ]:
# Running experiment using Estimator
# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/training-with-deep-learning/distributed-cntk-with-custom-docker/distributed-cntk-with-custom-docker.ipynb
from azureml.train.estimator import Estimator

params= {'--data_folder' : ds.as_mount() }

estimator = Estimator(source_directory='script',
                      compute_target=compute_target,
                      #compute_target='local',
                      entry_script='train.py',
                      script_params=params,
                      node_count=1,
                      process_count_per_node=1,
                      pip_requirements_file = '../docker/requirements.txt', # pip packages
                      custom_docker_image='<ACR_NAME>.azurecr.io/dw-tf-od1:v2', # using public docker hub
                      use_gpu=False)

In [ ]:
from azureml.core import Experiment

experiment_name = 'dw-exp-ssdv2lite'
experiment = Experiment(ws, name=experiment_name)

run = experiment.submit(estimator)
print(run)

run.wait_for_completion(show_output=True)

In [ ]:
#The trained model is stored from Experiment run in the Experiment -> Output+Logs tab. 

In [ ]:
trained_model_path = os.getcwd()+'/outputs'
print(trained_model_path)

In [ ]:
# Download the retrained frozen graph model (frozen_inference_graph.pb) and pipeline.config created from transfer learning experiment to the outputs folder. Time to download varies based on the size of the model.  
trained_model_path = os.getcwd()+'/outputs'
run.download_file(name = 'outputs/frozen_model/frozen_inference_graph.pb', output_file_path = trained_model_path)
run.download_file(name = 'outputs/frozen_model/pipeline.config', output_file_path = trained_model_path)
print(trained_model_path)

In [ ]:
'''
#Get the best model
minimum_rmse_runid = None
minimum_rmse = None

for run in experiment.get_runs():
    run_metrics = run.get_metrics()
    run_details = run.get_details()
    # each logged metric becomes a key in this returned dict
    run_rmse = run_metrics[""]
    run_id = run_details[""]

    if minimum_rmse is None:
        minimum_rmse = run_rmse
        minimum_rmse_runid = run_id
    else:
        if run_rmse < minimum_rmse:
            minimum_rmse = run_rmse
            minimum_rmse_runid = run_id

print("Best run_id: " + minimum_rmse_runid)
print("Best run_id rmse: " + str(minimum_rmse))
'''

In [ ]:
'''
from azureml.core import Run
best_run = Run(experiment=experiment, run_id=minimum_rmse_runid)
print(best_run.get_file_names())
'''

In [ ]:
#Reload workspace details for module twin update

from azureml.core import Workspace

ws = Workspace.from_config()
ws.get_details()

In [ ]:
#Register the trained model. Once register you'll find the model in the Models section on the left pane
from azureml.core.model import Model

model = Model.register(model_path = trained_model_path,
                      model_name = "Forks_Scissors_ssdv2lite",
                      tags = {"data": "ssd_mobilenetv2lite", "model": "object_detection", "type": "ssd_mobilenetv2lite"},
                      description = "Retrained Forks Scissors based on ssd_mobilenetv2lite",
                      workspace = ws)

In [ ]:
'''
### Convert the trained model to IR -> Blob using Intel Openvino toolkit for running on Devkit running Myriadx chipset
1) Setup Intel openvino toolkit on local machine https://docs.openvinotoolkit.org/latest/openvino_docs_install_guides_installing_openvino_windows.html
2) Download the Frozen_inference_graph.pb model file, pipeline.config, labels.txt and config.json from the notebook vm /outputs folder to local machine for model conversion
3) Model conversion to IR and Blob. Run the command from the command prompt as administrator
Pb->IR
python "c:\Program Files (x86)\IntelSWTools\openvino\deployment_tools\model_optimizer\mo_tf.py"  --input_model frozen_inference_graph.pb --tensorflow_object_detection_api_pipeline_config pipeline.config --transformations_config   "C:\Program Files (x86)\IntelSWTools\openvino\deployment_tools\model_optimizer\extensions\front\tf\ssd_v2_support.json" 

IR->Blob
"C:\Program Files (x86)\IntelSWTools\openvino_2020.3.194\deployment_tools\inference_engine\bin\intel64\Release\myriad_compile.exe" -m frozen_inference_graph.xml -o fast-rcnn-resnet50.blob -VPU_MYRIAD_PLATFORM VPU_MYRIAD_2480 -VPU_NUMBER_OF_SHAVES 8 -VPU_NUMBER_OF_CMX_SLICES 8 -iop "image_tensor:U8, image_info:FP32" -op FP32
4) If you trained with your custom data set, make sure to update the labels.txt with the classes used for training
5) Zip the converted model file .blob, labels.txt and config.json to model_ssdv2l.zip and upload the zip file to /ouputs/convertedmodel folder in notebook vm
6) Next step is to upload the zip file to blobstore and push it to the devkit using module twin update
'''

In [7]:
#Reload workspace details for training

from azureml.core import Workspace

ws = Workspace.from_config()
ws.get_details()

{'id': '/subscriptions/7e0a3a6c-0601-4e22-b727-e1c85bfaa678/resourceGroups/ADEResource_Tony/providers/Microsoft.MachineLearningServices/workspaces/dw-ws',
 'name': 'dw-ws',
 'location': 'westus',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'sku': 'Basic',
 'workspaceid': 'f6430f8f-646b-45d0-8037-74ca244a4d79',
 'description': '',
 'friendlyName': 'dw-ws',
 'creationTime': '2020-07-16T17:15:31.9177379+00:00',
 'containerRegistry': '/subscriptions/7e0a3a6c-0601-4e22-b727-e1c85bfaa678/resourceGroups/ADEResource_Tony/providers/Microsoft.ContainerRegistry/registries/dwws03f5f537',
 'keyVault': '/subscriptions/7e0a3a6c-0601-4e22-b727-e1c85bfaa678/resourcegroups/aderesource_tony/providers/microsoft.keyvault/vaults/dwwskeyvault681fff4468ba',
 'applicationInsights': '/subscriptions/7e0a3a6c-0601-4e22-b727-e1c85bfaa678/resourcegroups/aderesource_tony/providers/microsoft.insights/components/dwwsinsightsa9bd859def41',
 'identityPrincipalId': '60928247-c3de-49a6-8737-a13c47479cc0',
 '

In [8]:
# get the default datastore
ds = ws.get_default_datastore()
print(ds.name, ds.datastore_type, ds.account_name, ds.container_name)

workspaceblobstore AzureBlob dwwsstorage6f7c418cca1e4 azureml-blobstore-f6430f8f-646b-45d0-8037-74ca244a4d79


In [17]:
#set data path for model.zip
data_path = 'modelpath'
ds.upload(src_dir='./outputs/convertedmodel', target_path=data_path, overwrite=True)

Uploading an estimated of 1 files
Uploading ./outputs/convertedmodel/model_ssdv2l.zip
Uploaded ./outputs/convertedmodel/model_ssdv2l.zip, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_428e764474cf49d59e8e2d077a298b4c

In [18]:
#Generated Saas url for module twin update
from azure.storage.blob.baseblobservice import BaseBlobService,BlobPermissions
#from azure.storage.blob import BlobPermissions
from datetime import datetime, timedelta

AZURE_ACC_NAME = ds.account_name
AZURE_PRIMARY_KEY = ds.account_key
AZURE_CONTAINER = ds.container_name
AZURE_BLOB=ds.name
AZURE_File=data_path+'/model_ssdv2l.zip'

service = BaseBlobService(account_name=AZURE_ACC_NAME, account_key=AZURE_PRIMARY_KEY)
sas_url  = service.generate_blob_shared_access_signature(AZURE_CONTAINER,AZURE_File,permission=BlobPermissions.READ,expiry= datetime.utcnow() + timedelta(hours=48))
#sas_url = block_blob_service.generate_blob_shared_access_signature(AZURE_CONTAINER,AZURE_File,permission=BlobPermissions.READ,expiry= datetime.utcnow() + timedelta(hours=48))

#block_blob_service = BlockBlobService(account_name=AZURE_ACC_NAME, account_key=AZURE_PRIMARY_KEY)
#sas_url = block_blob_service.generate_blob_shared_access_signature(AZURE_CONTAINER,AZURE_File,permission=BlobPermissions.READ,expiry= datetime.utcnow() + timedelta(hours=48))
downloadurl ='https://'+AZURE_ACC_NAME+'.blob.core.windows.net/'+AZURE_CONTAINER+'/'+AZURE_File+'?'+sas_url
print('https://'+AZURE_ACC_NAME+'.blob.core.windows.net/'+AZURE_CONTAINER+'/'+AZURE_File+'?'+sas_url)
print(sas_url)

https://dwwsstorage6f7c418cca1e4.blob.core.windows.net/azureml-blobstore-f6430f8f-646b-45d0-8037-74ca244a4d79/modelpath/model_ssdv2l.zip?se=2020-08-01T06%3A10%3A33Z&sp=r&sv=2019-02-02&sr=b&sig=M%2Bc/43vTLztHHBH4nncgw52RQF/wW8%2Bzt8pUzwpGWfI%3D
se=2020-08-01T06%3A10%3A33Z&sp=r&sv=2019-02-02&sr=b&sig=M%2Bc/43vTLztHHBH4nncgw52RQF/wW8%2Bzt8pUzwpGWfI%3D


In [20]:
#Perform Module twin update
#Perform Module twin update
#Incorporate the connection string, device_id and the module_id values from your IoTHub

!pip install azure-iot-hub
import sys
from azure.iot.hub import IoTHubRegistryManager
from azure.iot.hub.models import Twin, TwinProperties

#Incorporate Iothub connection string and the default module name
#Go to Https://portal.azure.com
#Select your IoTHub
#Click on Shared access policies
#click service on right
#Copy the iothub connection string primary key

CONNECTION_STRING = "HostName=<IOTHUB CONNECTION STRING>"
DEVICE_ID = '<DEVICE_ID>'
MODULE_ID = "azureeyemodule"

try:
    # RegistryManager
    iothub_registry_manager = IoTHubRegistryManager(CONNECTION_STRING)

    module_twin = iothub_registry_manager.get_module_twin(DEVICE_ID, MODULE_ID)
    print ( "" )
    print ( "Module twin properties before update    :" )
    print ( "{0}".format(module_twin.properties) )

    # Update twin
    twin_patch = Twin()
    twin_patch.properties = TwinProperties(desired={"ModelZipUrl": downloadurl})
    updated_module_twin = iothub_registry_manager.update_module_twin(
        DEVICE_ID, MODULE_ID, twin_patch, module_twin.etag
    )
    print ( "" )
    print ( "Module twin properties after update     :" )
    print ( "{0}".format(updated_module_twin.properties) )

except Exception as ex:
    print ( "Unexpected error {0}".format(ex) )
except KeyboardInterrupt:
    print ( "IoTHubRegistryManager sample stopped" )


Module twin properties before update    :
{'additional_properties': {}, 'desired': {'ModelZipUrl': 'https://dwwsstorage6f7c418cca1e4.blob.core.windows.net/azureml-blobstore-f6430f8f-646b-45d0-8037-74ca244a4d79/modelpath/model_ssdv2l.zip?se=2020-08-01T06%3A10%3A33Z&sp=r&sv=2019-02-02&sr=b&sig=M%2Bc/43vTLztHHBH4nncgw52RQF/wW8%2Bzt8pUzwpGWfI%3D', 'Running': True, 'Logging': True, '$metadata': {'$lastUpdated': '2020-07-30T06:10:44.3138766Z', '$lastUpdatedVersion': 5, 'ModelZipUrl': {'$lastUpdated': '2020-07-30T06:10:44.3138766Z', '$lastUpdatedVersion': 5}, 'Running': {'$lastUpdated': '2020-07-30T05:22:07.7032238Z', '$lastUpdatedVersion': 1}, 'Logging': {'$lastUpdated': '2020-07-30T05:22:07.7032238Z', '$lastUpdatedVersion': 1}}, '$version': 5}, 'reported': {'$metadata': {'$lastUpdated': '2020-07-30T05:22:07.7032238Z', '$lastUpdatedVersion': 1}, '$version': 1}}

Module twin properties after update     :
{'additional_properties': {}, 'desired': {'ModelZipUrl': 'https://dwwsstorage6f7c418cca1

In [ ]:
# The trained model will get pushed to the IoT Edge device via module twin update method
# Check model inferencing by connecting monitor to the devkit or by installing VLC media player : 
#Install VLC from https://www.videolan.org/vlc/ and install on “Windows” to check the camera function of “Azure Eye”.

#Check video stream:
#1.	Select Media -> Open Network Stream…
#2.	Input the network stream: “rtsp://[ip of PE-101]:8554/result” then click “Play” button.

In [ ]:
#Environment Cleanup
#Check the docker image id created for the acr repo name
#!docker images

In [ ]:
#Delete docker registry by uncommenting the below step and including the image id
#!docker rmi -f <imageid>

In [ ]:
# delete cpu compute
"""
mycompute = AmlCompute(workspace=ws, name='dw-cpu1')
mycompute.delete()

# delete gpu compute
mycompute = AmlCompute(workspace=ws, name='dw-gpu')
mycompute.delete()
"""

In [ ]:
# delete workspace
#ws.delete(delete_dependent_resources=True)
